<h1> Largest Palindrome Product </h1>

A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is $9009 = 91 \times 99$. 

Find the _largest_ palindrome made from the _product_ of _two 3-digit numbers_.

The formal definition of a palindromic number in accordance to [wikipedia](https://en.wikipedia.org/wiki/Palindromic_number) is as follows. Consider a number $n>0$ in base $b\geq 2$,
$$
    n = \sum_{i=0}^k a_i b^i
$$
where $n$ is written in standard notation with $k+1$ digits $a_i$, $0 \leq a_i < b \forall i$ and $a_k \neq 0$. Then, $n$ is palindromic $\Leftrightarrow a_i = a_{k-i} \forall i$.
Assuming we are in base 10 (decimal system), then the equation is, 
$$
    n = \sum_{i=0}^k a_i 10^i \qquad \Longleftrightarrow \qquad a_i = a_{k-i} \forall i
$$

**Brute force method:** Start from $999 \times 998$ and test if each number is palindromic. The first one is the solution. Upperbound, need to test $900$ numbers. 

In [2]:
/// Function to test if number is palindromic
fn is_palindrome(num: Vec<usize>) -> bool {
    let max_n: usize = (num.len() as f64).sqrt().ceil() as usize;
    let vec_len: usize = num.len() as usize;
    
    for (i, n) in num.iter().enumerate(){
//         println!("{}, {}", i, n);
        if n != &num[(vec_len - i - 1) as usize] {
            return false
        } else if i == max_n {
            return true
        }
    }
    true
}

In [3]:
/// Function to convert a number into a vector containing its digits 
fn to_vec(num: usize) -> Vec<usize> {
    let num_str = num.to_string();
    let num_char = num_str.chars();
    let n = num_str.len();
    let mut out = vec![0 as usize; n];
    let mut i: usize = 0;
    
    for x in num_char {
        out[i] = x.to_digit(10_u32).unwrap() as usize;
        i += 1
    }
//     println!("{:?}", out);
    
    out
}

In [4]:
/// Function to find largest palindromic number
fn largest_palindromic_number() -> usize{
    for i in (101..1000).rev() {
        let n: usize = i*(i-1);
//         println!("{}", n);
//         println!("{}", i);
        let is_pal = is_palindrome(to_vec(n));
//         println!("{}", is_pal);
        if is_pal {
//             println!("{} {}", i, i-1);
            return n;
        }
    }
    0
}

In [5]:
let sol = largest_palindromic_number();
println!("{}", sol);

289982


This solution is **incorrect**. 

This is only the solutions along the leading diagonal of two numbers from $[999,101]$. To find all possible combinations, there is a need to find a 2D array with $900$ values in each dimension. Therefore, the actual number of multiples to evaluate is $900 \times 900$.

Therefore, the brute force method is not suitable. I knew it wouldn't be so easy.

**Hopefully smarter method**

Consider, what is the multiplication of two 3-digit values. It is, 
<!-- $$
(999 - x)(999-y) = 998001 - 999x - 999y + xy = 998001 - 999(x + y) + xy
$$  -->
$$
n = (999 - x)(999-y) = 998001 - 999(x + y) + xy
$$ 
where $0 \leq x \leq 900$ and $0 \leq y \leq 900$.
Alternatively, 
<!-- $$
(1000 - x)(1000-y) = 1000000 - 1000x - 1000y + xy = 1000000 - 1000(x+y) + xy
$$  -->
$$
n = (1000 - x)(1000-y) = 1000000 - 1000(x+y) + xy
$$ 
where $1 \leq x \leq 900$ and $1 \leq y \leq 900$.

For $999999 \leq n < 99999$, 
$$
    n_{pal} = a(100000) + b(10000) + c(1000) + d(100) + e(10) + f
$$
if the number is palindrome, then it fulfils that, 
$$
    n_{pal} = a(100001) + b(10010) + c(1100)
$$
where $a = n_{pal} \% 10$, $b = (n_{pal}/10) \% 10$, $c = (n_{pal}/100) \% 10$, i.e. $a$ is the digit in the ones place, etc.
N.B. $\% \implies$ modulo.

Therefore, if $n$ is used to find $a, b, c$ and the resulting number (using equation for $n_{pal}$) is the same (i.e. $n - n_{pal} = 0$). Then, the number is palindrome. 

This reduces the computational cost of each evaluation but it doesn't decrease the number of evaluations

<!-- If I equate $n = n_{pal}$,
$$
    100000(a-10) + 10010b + 1000(c + x + y) + 100c + a - xy = 0 
$$ -->
<!-- The condition also implies that $a-c \geq 0 \implies a \geq c$. Therefore,  -->

If $1000<xy<100$, then the first 3 numbers are identical to the last 3 numbers, 
\begin{gather}
    100a + 10 b + c = 1000-(x+y) \label{eq: first 3} \\
    100c + 10 b + a = xy \label{eq: last 3}
\end{gather}
Solving the equations simultaneously, Eqns (\ref{eq: first 3}) $-$ (\ref{eq: last 3}),
\begin{equation}
    99 (a-c) = 1000 - (x + y + xy) \label{eq: sim}
\end{equation}
Since the problem is symmetrical, $a-c$ will be within the range $[0,9]$. Using the identity, $(x + g)(y+h) = xy + xh + gy + gh$, Eqn \ref{eq: sim} can be rearrange to, 
\begin{equation}
    1001 - 99 (a-c) = (x+1)(y+1) \implies y+1 = \frac{1001 - 99 (a-c) }{x+1}
\end{equation}
Therefore, only $10 \times 1100=11000$ as opposed to $900^2= 810000$ (all $xy$ values between $1000<xy<100$). 

This is also significantly cheaper as only simple mathematical operations need to be performed

In [6]:
fn move_3_palindrome() -> usize {
    for z in 0..9 {
        let n: usize = 1001-99*z;
        for i in 2..n {
            // 1 <= x <= 900
            // Therefore, min(x+1) = 2
            if n % i == 0 {
                // Condition implies perfectly dividable => multiple 
                let x: usize = i - 1; 
                let y: usize = n/i  - 1;
                let eq1: usize = 1000 - 1*(x + y);
                let eq2: usize = x*y;
                if eq2 > 999 {
                    break;
                }
//                 println!("{}, {}", eq1, eq2);
                
                if (eq1%10) == (eq2/100)%10 && (eq1/10)%10 == (eq2/10)%10 && (eq1/100)%10 == eq2%10 {
//                     println!("{}, {}", x, y);
                    return (1000 - x)*(1000 - y)
                }
            }
        }
    }
    0
}

In [7]:
let soln = move_3_palindrome();
println!("{}", soln);

906609


I want a number larger than this. Therefore, I want $xy$ to be smaller.

Actually, for the remaining ones to test, it is feasible to test it using brute force. 

In [8]:
for x in 1..99 {
    for y in 1..99 {
        if x*y > 100 {
            break;
        }
        let n: usize = (1000-x)*(1000-y);
        let a: usize = n%10;
        let b: usize = (n/10)%10;
        let c: usize = (n/100)%10;
        let n_pal: usize = a*100001 + b*10010 + c*1100;
        if n == n_pal{
            println!("{}", n);
        }
    }
}
println!("A larger palindrome number does not exist");

A larger palindrome number does not exitst
